In [1]:
DATA_DIR = "/home/jinho/FlagEmbedding_Aizip/beir/data/sb_qna"

In [2]:
import json

with open(f'{DATA_DIR}/corpus.jsonl', 'r') as f:
    corpus = [json.loads(line) for line in f]

In [3]:
with open(f'{DATA_DIR}/test_queries.jsonl', 'r') as f:
    queries = [json.loads(line) for line in f]

In [4]:
with open(f'{DATA_DIR}/test_qrels.jsonl', 'r') as f:
    qrels = [json.loads(line) for line in f]

In [5]:
SEARCH_RESULT_FPATH = "/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/sb_qna/20250224_1739/alpha_0.9_rank_4/multilingual-e5-large/bge-reranker-v2-m3/sb_qna-test.json"

In [6]:
with open(SEARCH_RESULT_FPATH, "r") as f:
    search_results = json.load(f)

In [7]:
results = search_results['search_results']

In [8]:
qrel_dic = {}
for d in qrels:
    qrel_dic[d['qid']] = d['docid']

In [9]:
corpus_dic = {}
for c in corpus:
    corpus_dic[c['id']] = c['text']

In [10]:
correct = []
incorrect = []
for q in queries:
    qid = q['id']
    qtext = q['text']
    gt_corpus = qrel_dic[qid]
    retrieved_corpus = list(results[qid].keys())
    if gt_corpus in retrieved_corpus:
        correct.append(qid)
    else:
        context = ""
        for i in range(len(retrieved_corpus)):
            c = retrieved_corpus[i]
            context += f"CONTEXT {i}\n\n"
            context += corpus_dic[c]
            context += "\n\n"
        incorrect.append({"input": qtext, "gt_corpus": corpus_dic[gt_corpus], "retrieved": context})

In [11]:
print(incorrect[0]['retrieved'])

CONTEXT 0

「Google Choromeクラッシュしました。このバルーンをクリックして再起動してください」が表示された場合は下記マニュアルを参照し、再インストールを行って下さい。
・Google Chrome再インストール方法
 http://oa.bb.local/curator_web/oasupport/?jumpurl=http://oa.bb.local/oasupport/%3Fpost_type%3Dpost%26p%3D35747

CONTEXT 1

Zoom Chrome Extension（旧：Zoom Scheduler）の起動後にLoadingと表示される場合は、下記マニュアルの手順でChromeのバージョンアップを実施下さい。

・Google Chrome再インストール方法
http://oa.bb.local/curator_web/oasupport/?jumpurl=http://oa.bb.local/oasupport/%3Fpost_type%3Dpost%26p%3D35747
※ソフトウェアセンターが利用できない場合は、付録1・付録2の手順で対応下さい
※付録2-2では最新バージョンをダウンロード下さい

CONTEXT 2

GoogleChromeが消失した場合、下記手順にてGoogleChromeを再インストール願います。

・Google Chrome再インストール方法
 http://oa.bb.local/curator_web/oasupport/?jumpurl=http://oa.bb.local/oasupport/%3Fpost_type%3Dpost%26p%3D35747

ソフトウェアセンターからインストール不可の場合は、下記FAQを参考に再インストールを実施して下さい。
参考FAQ
・Chromeのバージョンアップをしたら、Chromeが消えてしまった［FAQ No：F000200004］
 http://oa.bb.local/curator_web/oasupport/?jumpurl=http://oa.bb.local/oasupport/faq/F000200004

CONTEXT 3

OA側から配布しているChromeについては、正常動作を確認した上で配布してお

In [12]:
len(correct) / (len(correct) + len(incorrect))

0.833789007259357

In [13]:
import csv

In [16]:
import os

In [17]:
dirname = os.path.dirname(SEARCH_RESULT_FPATH)

In [18]:
dirname

'/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/sb_qna/20250224_1739/alpha_0.9_rank_4/multilingual-e5-large/bge-reranker-v2-m3'

In [19]:
# Write to CSV file
with open(f'{dirname}/missed_retrieval.csv', 'w', newline='') as csvfile:
    # Get the field names from the first dictionary
    fieldnames = incorrect[0].keys()
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write header
    writer.writeheader()
    
    # Write rows
    writer.writerows(incorrect)

print("CSV file written successfully!")

CSV file written successfully!
